# Full example: energy demand using adaptive setpoint temperatures in Japan

In this notebook, we're going to run a simulation with adaptive setpoint temperatures based on ASHRAE 55, and we are going to compare these with the static setpoints from the COOLBIZ campaign. Also, we are going to run simulations considering the ASHRAE 55 in NV mode, to compare it with MM. We are going to run some simulations  in Sapporo and Naha (some Japanese locations), and also we are going to analyse and visualize the data. This Jupyter Notebook partially replicates the research paper _Extending the use of adaptive thermal comfort to air-conditioning: the case study of a local Japanese comfort model in present and future scenarios_[1].

[1] Sánchez-García, D., Bienvenido-Huertas, D., Pulido-Arcas, J.A., Rubio-Bellido, C., 2023. Extending the use of adaptive thermal comfort to air-conditioning: The case study of a local Japanese comfort model in present and future scenarios. Energy Build. 285, 112901. https://doi.org/10.1016/j.enbuild.2023.112901

First of all, given EnergyPlus 23.1 is installed in the default path (C:\EnergyPlusV23-1-0), and accim 0.6.11 has been installed by entering 'pip install accim==0.6.11' in the CMD dialog, let's prepare the files we need: the IDF(s) and the EPW(s). Let's see what file we have in the folder and then we'll continue with the IDF(s).

First of all, let's store the input files names so that we can re-run this notebook.

In [3]:
from os import listdir
input_files = [i for i in listdir()]

## 1. IDF (using `addAccis()`)

Say we have one or multiple IDF files, with an existing HVAC system (in this case, the use of mixed-mode ScriptType 'ex_mm' is not recommended; only full air-conditioning) or with no HVAC system at all (in this case, any of the 'vrf_ac' or 'vrf_mm' ScriptTypes are recommended). In this example, we are going to use an IDF without HVAC system, and we are going to use 'vrf_mm' so that accim adds a generic VRF system.

Let's see what IDFs we do have in our folder:

In [4]:
input_idfs = [i for i in listdir() if i.endswith('.idf')]
print(input_idfs)

['TestModel.idf']


So now, we're going to generate building energy models with setpoint temperatures based on the static setpoints for Japan (i.e. ComfStand takes the value 3) and ASHRAE 55 (i.e. ComfStand takes the value 2). We're going to select the 80% acceptability levels for both (i.e. CAT takes the value 80), and we're going to select the setpoint behaviour to horizontally extend the setpoint temperatures (or comfort limits) when applicability limits are exceeded (i.e. ComfMod takes the value 3). There are 2 methods to apply adaptive setpoint temperatures:
- Short method, which is running the following to lines of code:
```
from accim.sim import accis
accis.addAccis()
```

When we run the 2 lines of code above, accim is going to ask us to enter some information it needs to generate the output IDFs. The data we're going to input, in the same order, is:
- Enter the ScriptType: **vrf_mm**
- Enter the SupplyAirTempInputMethod: **temperature difference**
- Do you want to keep the existing outputs (true or false)?: **false**
- Enter the Output type (standard, simplified or detailed): **standard**
- Enter the Output frequencies separated by space (timestep, hourly, daily, monthly, runperiod): **hourly**
- Enter the EnergyPlus version (9.1 to 23.1): **23.1**
- Enter the Temperature Control method (temperature or pmv): **temperature**

After that, accim will let us know the information we have entered, and it will start the generic IDF generation process. Lots of actions are going to be performed, and all of them will be printed on screen. Once this process is done, accim will let us know if any of the IDFs is not going to work for any reason, and then it will start the output IDF files generation process. Then, accim will ask us again to enter some information, this time to generate the output IDF(s). The data we are going to enter now is:

- Enter the Comfort Standard numbers separated by space: **2 3**
- Enter the Category numbers separated by space: **80**
- Enter the Comfort Mode numbers separated by space: **0 3** (where 0 and 3 are respectively static and adaptive setpoints)
- Enter the HVAC Mode numbers separated by space: **1 2** (in this case we have also selected 1 for naturally ventilated, to see the difference with mixed-mode)
- Enter the Ventilation Control numbers separated by space: **0**

For all the remaining arguments, we're going to hit enter to omit it and take the default value. Finally, accim will let us know the list of output IDFs and will ask for confirmation to proceed:

- Do you still want to run ACCIS? [y/n]: **y**

Alternatively, we could specify all the arguments when calling the function, as shown in the cell below:

In [5]:
from accim.sim import accis
accis.addAccis(
    ScriptType='vrf_mm',
    SupplyAirTempInputMethod='temperature difference',
    Output_keep_existing=False,
    Output_type='standard',
    Output_freqs=['hourly'],
    EnergyPlus_version='23.1',
    TempCtrl='temperature',
    ComfStand=[2, 3],
    CAT=[80],
    ComfMod=[0, 3],
    HVACmode=[1, 2],
    VentCtrl=[0],
    VSToffset=[0],
    MinOToffset=[50],
    MaxWindSpeed=[50],
    ASTtol_steps=0.1,
    ASTtol_start=0.1,
    ASTtol_end_input=0.1,
    confirmGen=True
)


--------------------------------------------------------
Adaptive-Comfort-Control-Implemented Model (ACCIM)
--------------------------------------------------------

This tool allows to apply adaptive setpoint temperatures. 
For further information, please read the documentation: 
https://accim.readthedocs.io/en/master/
For a visual understanding of the tool, please visit the following jupyter notebooks:
-    Using addAccis() to apply adaptive setpoint temperatures
https://github.com/dsanchez-garcia/accim/blob/master/accim/sample_files/jupyter_notebooks/addAccis/using_addAccis.ipynb
-    Using rename_epw_files() to rename the EPWs for proper data analysis after simulation
https://github.com/dsanchez-garcia/accim/blob/master/accim/sample_files/jupyter_notebooks/rename_epw_files/using_rename_epw_files.ipynb
-    Using runEp() to directly run simulations with EnergyPlus
https://github.com/dsanchez-garcia/accim/blob/master/accim/sample_files/jupyter_notebooks/runEp/using_runEp.ipynb
-    

IDF has been saved
Ending with file:
TestModel

=======================END OF GENERIC IDF FILE GENERATION PROCESS=======================

The following IDFs will not work, and therefore these will be deleted:
None

=======================START OF OUTPUT IDF FILES GENERATION PROCESS=======================

The list of output IDFs is going to be:
TestModel[CS_INT ASHRAE55[CA_80[CM_0[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_INT ASHRAE55[CA_80[CM_0[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_INT ASHRAE55[CA_80[CM_3[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_INT ASHRAE55[CA_80[CM_3[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_JPN Rijal[CA_80[CM_0[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_JPN Rijal[CA_80[CM_0[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_JPN Rijal[CA_80[CM_3[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_JPN Rijal[CA_80[CM_3[HM_2[VC_0[VO_0.0[MT_50.0[M

So, now let's see the list of output IDFs we have generated

In [6]:
output_idfs = [i for i in listdir() if i.endswith('.idf') and i not in input_idfs]
print(*output_idfs, sep='\n')

TestModel[CS_INT ASHRAE55[CA_80[CM_0[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_INT ASHRAE55[CA_80[CM_0[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_INT ASHRAE55[CA_80[CM_3[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_INT ASHRAE55[CA_80[CM_3[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_JPN Rijal[CA_80[CM_0[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_JPN Rijal[CA_80[CM_0[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_JPN Rijal[CA_80[CM_3[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_JPN Rijal[CA_80[CM_3[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf


In this case, we have generated more IDFs than we need, so let's remove the others. We only want a single naturally ventilated IDF, to compare the indoor temperature with the mixed-mode IDF with adaptive setpoints. IDFs are NV when HVACmode takes the value 1.

In [7]:
idfs_to_keep = [
    'TestModel[CS_INT ASHRAE55[CA_80[CM_3[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf',
    'TestModel[CS_INT ASHRAE55[CA_80[CM_3[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf',
    'TestModel[CS_JPN Rijal[CA_80[CM_0[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf'
]
idfs_to_be_removed = [i for i in listdir() if i.endswith('.idf') and i not in idfs_to_keep and i not in input_idfs]
print(*idfs_to_be_removed, sep='\n')

TestModel[CS_INT ASHRAE55[CA_80[CM_0[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_INT ASHRAE55[CA_80[CM_0[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_JPN Rijal[CA_80[CM_0[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_JPN Rijal[CA_80[CM_3[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_JPN Rijal[CA_80[CM_3[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf


In [8]:
from os import remove
for i in idfs_to_be_removed:
    remove(i)

Let's see what IDFs we do finally have:

In [9]:
output_idfs = [i for i in listdir() if i.endswith('.idf') and i not in input_idfs]
print(*output_idfs, sep='\n')

TestModel[CS_INT ASHRAE55[CA_80[CM_3[HM_1[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_INT ASHRAE55[CA_80[CM_3[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf
TestModel[CS_JPN Rijal[CA_80[CM_0[HM_2[VC_0[VO_0.0[MT_50.0[MW_50.0[AT_0.1[NS_X.idf


So, we're done with the IDFs. You can see these have been named based on the input data, separated by the character '['. Let's move to the EPWs.

## 2. EPW (using `rename_epw_files()`)

Let's see the EPWs we are going to use for the simulations:

In [14]:
original_epws = [i for i in listdir() if i.endswith('.epw')]
print(*original_epws, sep='\n')

Current_Naha_JA-hour.epw
Current_Sapporo_JA-hour.epw
rcp85_2100_Naha_JA-hour.epw
rcp85_2100_Sapporo_JA-hour.epw


However, we don't want to run the simulations using that name. To ease the later data analysis, we are going to rename the EPW files following the pattern 'Country_City_RCPscenario-Year'. This way, data will be able to be grouped by country, city, RCP scenario and year. So let's rename them running the code in the cell below.

First, accim will try to rename them based on the original name and the geolocation. If no match between those is found, accim will assign the string 'UNKNOWN' to the city. Then, accim will ask you if you want to edit some of the new names. If so, you'll need to enter the IDs:
- If any of the city or subcountry names needs some amendment (if you are not happy with any of the available options, you can exclude it from renaming at the next stage), please enter the EPW IDs separated by space:**(hit enter)**

Afterwards, you'll be asked to enter the new city name for each ID you previously entered (in this case, 0 1 2 3). So, 
- Regarding the file ID: 0 ... Please enter the amended city or subcountry, which must be unique: **Naha**
- Regarding the file ID: 1 ... Please enter the amended city or subcountry, which must be unique: **Sapporo**
- Regarding the file ID: 2 ... Please enter the amended city or subcountry, which must be unique: **Naha**
- Regarding the file ID: 3 ... Please enter the amended city or subcountry, which must be unique: **Sapporo**

Then, accim will let you know the old names, and the new named after amendments. Next, accim will ask you if you want to exclude some EPW from renaming. In this case, we're just going to hit enter to continue because we don't want to exclude any:
- If you want to exclude some EPWs from renaming, please enter the new names separated by space, otherwise, hit enter to continue:

Finally, accim will ask for confirmation to proceed with the renaming:
Do you want to rename the file or files? [y/n]:**y**

At this point, accim will make a copy of the EPWs and rename them. Afterwards, we would be asked if we want to delete the older EPWs. In this case, we won't because the deletion has been already set to False in the arguments.

In [ ]:
from accim.data.data_preprocessing import rename_epw_files
rename_epw_files(
    rename_dict={
        'Naha': 'Naha',
        'Sapporo': 'Sapporo'
    },
    confirm_deletion=False
)

Since no match has been found between RCP scenario Year and EPW file name, Present year has been assigned to the following EPW files:
Current_Naha_JA-hour.epw
Current_Sapporo_JA-hour.epw
The geolocation process has taken: 1.71 seconds (0.43 s/EPW)

The previous and new names of the EPW files and their unique IDs are:
ID: 0 / Current_Naha_JA-hour / Japan_UNKNOWN_Present
ID: 1 / Current_Sapporo_JA-hour / Japan_UNKNOWN_Present
ID: 2 / rcp85_2100_Naha_JA-hour / Japan_UNKNOWN_RCP85-2100
ID: 3 / rcp85_2100_Sapporo_JA-hour / Japan_UNKNOWN_RCP85-2100

Duplicates have been found in the renamed EPW files, therefore these need to be amended in the next stage:
['Japan_UNKNOWN_RCP85-2100', 'Japan_UNKNOWN_Present']

"UNKNOWN" city or subcountry have been found in the renamed EPW files, therefore these need to be amended in the next stage.


Now, let's see what EPWs we do have:

In [12]:
all_epws = [i for i in listdir() if i.endswith('.epw')]
print(*all_epws, sep='\n')

Japan_Naha_Present.epw
Japan_Naha_RCP85-2100.epw
Japan_Sapporo_Present.epw
Japan_Sapporo_RCP85-2100.epw


We can see the new EPWs are:

In [13]:
new_epws = [i for i in listdir() if i.endswith('.epw') if i not in original_epws]
print(*new_epws, sep='\n')

EPWs are correctly renamed, so now let's move the old EPWs to a different folder to save them as a backup.

In [ ]:
import shutil
for i in original_epws:
    shutil.move(i, f'backup/{i}')

Now, we can move to the next stage.

## 3. Running the simulation (using `runEp()`)

At this point, we have prepared the IDF(s) we are going to simulate, which are

In [ ]:
print(*output_idfs, sep='\n')

as well as the locations where we are going to run those simulations, whose EPWs are:

In [ ]:
print(*new_epws, sep='\n')

So, we are going to simulate all IDF(s) with all EPW(s). When we run later the simulations using accim, the output files (i.e. the CSVs) will be named following the pattern **'idf[epw'**, where the character '[' is used as a separator for later data analysis, so that CSV rows can be grouped by EPW location. You may have noticed the same character is used as a separator in the IDF name, in order to group the CSV rows depending on the input data.

To run the simulations, 2 methods can be used:
- the shorter, in which the following 2 lines of code needs to be run:
    ```
    from accim.run import run
    run.runEp()
    ```
   After this, you'll be asked to enter the EnergyPlus version (which should coincide with the IDF EnergyPlus version):
    - Please enter the desired EnergyPlus version: **23.1**
    
   Then, you will need to say if you want to run only output IDFs of accim, or otherwise all existing IDFs in the folder:
    - Do you want to run only ACCIM output IDFs? [y or n]: **y**
    
   Next, accim will tell you the IDF(s) and EPW(s) it's going to use for the simulations, and finally all the simulations it's going to run based on the name pattern 'idf[epw'.
   Finally, it will ask for confirmation to proceed with the simulation:
    - Do you still want to proceed? [y or n]:**y**
- the longer method, in which the parameters are specified when calling the function. We'll use the longer method, so let's run the cell below. Since there are a few simulations, it might take a few minutes.

In [ ]:
from accim.run import run
run.runEp(
    runOnlyAccim=True, #only runs output IDFs, that is, IDFs with "[" in its name
    confirmRun=True, #to skip confirmation
    num_CPUs=4, #to specify the number of CPUs to be used
    EnergyPlus_version='23.1', #to specify the EnergyPlus version of the IDF, and the version of EnergyPlus you are going to run
)

So simulations are done. Let's see the CSV data we have now:

In [ ]:
csvs = [i for i in listdir() if i.endswith('.csv') and 'Zsz.csv' not in i and 'Table.csv' not in i]
print(*csvs, sep='\n')

Now, we can move to the last stage, in which data will be analysed and visualized.

## 4. Analysing and visualising the data (using `Table()`)

In order to analyse and visualize the data, we need to make a pandas DataFrame out of the CSVs. We will do this by using the `Table()` method. To use this method, a minimum knowledge and experience with Python programming is needed, so if this is not your case, you may struggle to make it work.

Let's create an hourly dataframe, since firstly we are going to compare indoor temperature with and without adaptive setpoint temperatures.

In [ ]:
from accim.data.data_postprocessing import Table
dataset_hourly = Table(
    #datasets=list #Since we are not specifying any list, it will use all available CSVs in the folder
    source_frequency='hourly', # This lets accim know which is the frequency of the input CSVs. Input CSVs with multiple frequencies are also allowed. It can be 'hourly', 'daily', 'monthly' and 'runperiod'. It can also be 'timestep' but might generate errors.
    frequency='hourly', # If 'daily', accim will aggregate the rows in days. It can be 'hourly', 'daily', 'monthly' and 'runperiod'. It can also be 'timestep' but might generate errors.
    frequency_agg_func='sum', #this makes the sum or average when aggregating in days, months or runperiod; since the original CSV frequency is in hour, it won't make any aeffect
    standard_outputs=True, 
    level=['building'], # A list containing the strings 'block' and/or 'building'. For instance, if ['block', 'building'], accim will generate new columns to sum up or average in blocks and building level.
    level_agg_func=['sum', 'mean'], # A list containing the strings 'sum' and/or 'mean'. For instance, if ['sum', 'mean'], accim will generate the new columns explained in the level argument by summing and averaging.
    level_excluded_zones=[],
    split_epw_names=True, #to split EPW names based on the pattern Country_City_RCPscenario-Year
)

So let's take a look at the DataFrame instance 'df' we have stored within Table instance 'dataset_hourly'.

In [ ]:
dataset_hourly.df

And the shape is:

In [ ]:
dataset_hourly.df.shape

Let's filter the columns we are going to use:

In [ ]:
dataset_hourly.format_table(
    type_of_table='custom', # Used to choose some predefined tables. It can be 'energy demand', 'comfort hours', 'temperature', 'all' or 'custom'
    custom_cols=[ #if type_of_table is 'custom', custom_cols is used to filter the desired columns to keep
        'Adaptive Cooling Setpoint Temperature_No Tolerance (°C)',
        'Adaptive Heating Setpoint Temperature_No Tolerance (°C)',
        'Building_Total_Zone Operative Temperature (°C) (mean)',
        'BLOCK1:ZONE2_ASHRAE 55 Running mean outdoor temperature (°C)',
        'Building_Total_Cooling Energy Demand (kWh/m2) (summed)',
        'Building_Total_Heating Energy Demand (kWh/m2) (summed)',
        'Building_Total_AFN Zone Infiltration Air Change Rate (ach) (summed)'
    ]
)

Again, let's take a look at the filtered df and the new shape:

In [ ]:
dataset_hourly.df

In [ ]:
dataset_hourly.df.shape

### 4.1 Visualizing the data 

At this point, the user might have a vague idea of what table can be done with the available data, especially if there is a large number of categorical variables. In order to get a clearer idea of this, the user can call the method named gather_vars_query. It only takes one argument, named vars_to_gather, which should be a list of the variables to be combined.

In this case, when IDFs with adaptive setpoint were generated, the arguments where more than one than one value were requested were ComfStand, ComfMod and HVACmode (ComfStand=[2, 3], ComfMod=[0, 3], HVACmode=[1, 2]). Therefore, these are all the categorical variables that change regarding the IDFs and the possibilities that might be interesting to study, and therefore the variables that have been entered in vars_to_gather.

Then, after calling gather_vars_query, accim prints on screen the categorical variables that contains more than one different value (i.e. ComfMod, since values are “CM_0” and “CM_3”), and the different combinations based on the combined variables, joined by character “[“:

In [ ]:
dataset_hourly.gather_vars_query(['ComfStand', 'ComfMod', 'HVACmode'])

And now, let's generate the figure data (a list of lists and dictionaries with all information to be plotted) with `generate_fig_data()` and afterwards, let's plot the figure with `scatter_plot()`.

In [ ]:

dataset_hourly.scatter_plot(
    vars_to_gather_rows=['EPW'], # variables to gather in rows of subplots
    vars_to_gather_cols=['ComfStand', 'ComfMod', 'HVACmode'],# variables to gather in columns of subplots; all categorical columns which have more than 1 different value across the rows, must be specified in this argument, otherwise you'll get an error.
    detailed_cols=['CS_INT ASHRAE55[CM_3[HM_1', 'CS_INT ASHRAE55[CM_3[HM_2'], # a list of the specific combinations of arguments to be plotted joined by [
    data_on_x_axis='BLOCK1:ZONE2_ASHRAE 55 Running mean outdoor temperature (°C)', #column name (string) for the data on x axis
    data_on_y_main_axis=[ #list which includes the name of the axis on the first place, and then in the second place, a list which includes the column names you want to plot
        [
            'Indoor Operative Temperature (°C)',
            [
                'Adaptive Cooling Setpoint Temperature_No Tolerance (°C)',
                'Adaptive Heating Setpoint Temperature_No Tolerance (°C)',
                'Building_Total_Zone Operative Temperature (°C) (mean)',
            ]
        ],
    ],
    colorlist_y_main_axis=[
        [
            'Indoor Operative Temperature (°C)',
            [
                'b',
                'r',
                'g',
            ]
        ],
    ],
    supxlabel='Running Mean Outdoor Temperature (°C)', # data label on x axis
    figname=f'Scatterplot_NV_vs_MM',
    figsize=6,
    ratio_height_to_width=0.33,
    confirm_graph=True
)

In this figure, you can see on the left column the simulations with free-running (or naturally ventilated) mode, while on the right, the same simulations using mixed-mode with adaptive setpoint temperatures, which introduce all hourly indoor temperatures within the adaptive thermal comfort limits.

Next, let's compare the indoor temperatures of ASHRAE 55 and the static setpoints for Japan, and in this case, we're also going to plot the hourly energy demand on the main y-axis.

In [ ]:
dataset_hourly.scatter_plot(
    vars_to_gather_cols=['ComfStand', 'ComfMod', 'HVACmode'], # variables to gather in rows of subplots
    vars_to_gather_rows=['EPW'],# variables to gather in columns of subplots
    detailed_cols=['CS_JPN Rijal[CM_0[HM_2', 'CS_INT ASHRAE55[CM_3[HM_2'], #we only want to see those combinations
    data_on_x_axis='BLOCK1:ZONE2_ASHRAE 55 Running mean outdoor temperature (°C)', #column name (string) for the data on x axis
    data_on_y_sec_axis=[ #list which includes the name of the axis on the first place, and then in the second place, a list which includes the column names you want to plot
        [
            'Air renovation (ach)',
            [
                'Building_Total_AFN Zone Infiltration Air Change Rate (ach) (summed)'
            ]
        ],
        [
            'Operative Temperature (°C)',
            [
                'Adaptive Cooling Setpoint Temperature_No Tolerance (°C)',
                'Adaptive Heating Setpoint Temperature_No Tolerance (°C)',
                'Building_Total_Zone Operative Temperature (°C) (mean)',
            ]
        ],
    ],
    data_on_y_main_axis=[ # similarly to above, a list including the name of the secondary y-axis and the column names you want to plot in it
        [
            'Energy (kWh/m2)',
            [
                'Building_Total_Cooling Energy Demand (kWh/m2) (summed)',
                'Building_Total_Heating Energy Demand (kWh/m2) (summed)',
            ]
        ],

    ],
    colorlist_y_sec_axis=[
        [
            'Air renovation (ach)',
            [
                'yellow'
            ]
        ],
        [
            'Operative Temperature (°C)',
            [
                'b',
                'r',
                'g',
            ]
        ],
    ],
    colorlist_y_main_axis=[
        [
            'Energy (kWh/m2)',
            [
                'cyan',
                'orange',
            ]
        ],
    ],
    supxlabel='Running Mean Outdoor Temperature (°C)', # data label on x axis
    figname=f'scatterplot_JPN_stat_ASH_adap_JPN_adap',
    figsize=6,
    ratio_height_to_width=0.33,
    confirm_graph=True
)

### 4.2 Analysing the data

Now, let's see how many comfort hours were considering the NV mode, and afterwards the MM considering ASHRAE 55, as well as the impact on energy demand. Since we want to see the runperiod totals, we will need to make a new instance of Table(), asking for runperiod frequency this time.

In [ ]:
from accim.data.data_postprocessing import Table
dataset_runperiod = Table(
    #datasets=list #Since we are not specifying any list, it will use all available CSVs in the folder
    source_frequency='hourly', # This lets accim know which is the frequency of the input CSVs. Input CSVs with multiple frequencies are also allowed. It can be 'hourly', 'daily', 'monthly' and 'runperiod'. It can also be 'timestep' but might generate errors.
    frequency='runperiod', # If 'daily', accim will aggregate the rows in days. It can be 'hourly', 'daily', 'monthly' and 'runperiod'. It can also be 'timestep' but might generate errors.
    frequency_agg_func='sum', #this makes the sum or average when aggregating in days, months or runperiod; since the original CSV frequency is in hour, it won't make any aeffect
    standard_outputs=True, 
    level=['building'], # A list containing the strings 'block' and/or 'building'. For instance, if ['block', 'building'], accim will generate new columns to sum up or average in blocks and building level.
    level_agg_func=['sum', 'mean'], # A list containing the strings 'sum' and/or 'mean'. For instance, if ['sum', 'mean'], accim will generate the new columns explained in the level argument by summing and averaging.
    level_excluded_zones=[],
    split_epw_names=True, #to split EPW names based on the pattern Country_City_RCPscenario-Year
)

dataset_runperiod.format_table(
    type_of_table='custom',
    custom_cols=[
        'Building_Total_Comfortable Hours_No Applicability (h) (mean)',
        'Building_Total_Total Energy Demand (kWh/m2) (summed)'
    ]
)

dataset_runperiod.wrangled_table(
    reshaping='unstack',
    vars_to_gather=['ComfStand', 'ComfMod', 'HVACmode'],
    baseline='CS_INT ASHRAE55[CM_3[HM_2',
    comparison_mode=['baseline compared to others'],
    comparison_cols=['absolute'],
    rename_dict={
        'CS_INT ASHRAE55[CM_3[HM_1': 'ASHRAE55_NV',
        'CS_INT ASHRAE55[CM_3[HM_2': 'ASHRAE55_MM',
        'CS_JPN Rijal[CM_0[HM_2': 'JPN_Stat_MM'
    }
)

dataset_runperiod.wrangled_df_unstacked

The table above shows us the comfort hours in NV (CS_INT ASHRAE55[CM_3[HM_1) mode ranges between 5111.58 and 6940.58 hours, while the same comfort model in mixed-mode with adaptive setpoints (CS_INT ASHRAE55[CM_3[HM_2) ranges between 8598.15 and 8758.85. Since there is no HVAC system in NV mode, the energy consumption is 0. With adaptive setpoints, the hvac energy consumption ranges between 265.82 and 419.02 (kWh/m2·year).

Now, we could finally export this table to Excel format for later style edition. Since the reshaping argument we used in the `wrangled_table()` method was 'unstack', the dataframe we are looking for to be exported is `dataset_runperiod.wrangled_df_unstacked`. If we used the 'pivot' argument, the dataframe would have been `dataset_runperiod.wrangled_df_pivoted`. So let's export it:

In [ ]:
dataset_runperiod.wrangled_df_unstacked.to_excel('df_unstacked.xlsx')

In [ ]:
import pandas as pd

df = pd.read_excel('df_unstacked.xlsx', index_col=[0, 1, 2], header=[0, 1])
df

Finally, so that we can run this jupyter notebook again, let's leave everything as it was at the beginning.

In [ ]:
for i in original_epws:
    shutil.move(f'backup/{i}', i)

In [ ]:
files_to_delete = [i for i in listdir() if i not in input_files]
print(*files_to_delete, sep='\n')

In [ ]:
for i in files_to_delete:
    remove(i)